In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:
import pandas as pd
#path = f"/home/lucas/Dropbox/SCIENCE/FREIWALD_LAB/DATA/DFallpa_Diego_230929_RULE_BASE_stroke.pkl"
animal = "Diego"
date = "230629"
path= f"/home/kgg/Desktop/eyetracking_analyses/{animal}-{date}/dfallpa.pkl"
DFallpa = pd.read_pickle(path)

In [4]:
DFallpa

,which_level,event,bregion,twind,pa,pa_x_shape
0,flex,fixon_preparation,M1_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(22, 896, 80)"
1,flex,fixon_preparation,M1_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(17, 896, 80)"
2,flex,fixon_preparation,PMv_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(39, 896, 80)"
3,flex,fixon_preparation,PMv_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(31, 896, 80)"
4,flex,fixon_preparation,PMd_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(24, 896, 80)"
5,flex,fixon_preparation,PMd_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(15, 896, 80)"
6,flex,fixon_preparation,dlPFC_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(27, 896, 80)"
7,flex,fixon_preparation,vlPFC_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(29, 896, 80)"
8,flex,fixon_preparation,vlPFC_a,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(17, 896, 80)"
9,flex,fixon_preparation,FP_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(15, 896, 80)"


In [5]:
# hack until resave dfallpa.pkl (added to main code)
for i in range(0,15):
    DFallpa.iloc[i].pa.Xlabels["trials"]["is-first-macrosaccade"] = DFallpa.iloc[i].pa.Xlabels["trials"]["shape-macrosaccade-index"]==1

In [ ]:
# for each trial, x=pc1, y=pc2
# reproduce figure

twind_list = [(-0.2, 0), (0, 0.2)] # relative to event
tbin_dur = 0.1 # windowing
tbin_slide = tbin_dur
reshape_method="trials_x_chanstimes"
pca_reduce=True
npcs_keep_force=6
#twind_overall=twind_list[0]

nbr = DFallpa.shape[0]
fig, axes = plt.subplots(nbr, 2, squeeze=False, figsize=(10, 5*nbr))
# then, can try plotting coding axis
test = None
for i in range(0, nbr):    
    pa = DFallpa.iloc[i]['pa']
    br = DFallpa.iloc[i]['bregion']
    #print(br)
    
    for twind in twind_list:
        twind_i = twind_list.index(twind)
        X, PAfinal, _, _, _ = pa.dataextract_state_space_decode_flex(twind_overall=twind, tbin_dur=tbin_dur, tbin_slide=tbin_slide,
                                                              reshape_method=reshape_method, pca_reduce=pca_reduce,
                                                              npcs_keep_force=npcs_keep_force)
        # get the values of the top 2 dimensions and plot
        scatter_x = np.array([list[0] for list in X])
        scatter_y = np.array([list[1] for list in X])
        n = PAfinal.Xlabels['trials']['shape-fixation']

        ax = axes[i,twind_i]
        #ax.scatter(x,y)
        ax.set_xlabel("PC1")
        ax.set_ylabel("PC2")
        ax.set_title(br+str(twind))
        ax.set_box_aspect(1)
        for g in np.unique(n):
            inds = np.where(n == g)
            ax.scatter(scatter_x[inds], scatter_y[inds], label=g)
plt.show()

In [7]:
## Cross-temporal decoder, trained on seqc_0_shape, tested on shape-fixation

from neuralmonkey.analyses.decode_good import decodewrapouterloop_categorical_cross_time_cross_var, decodewrapouterloop_categorical_cross_time_plot

SAVEDIR_ANALYSIS = f"/tmp/{animal}-{date}-decode"
SAVEDIR = f"{SAVEDIR_ANALYSIS}/4_cross_temporal_diff_var_train_test"
os.makedirs(SAVEDIR, exist_ok=True)

time_bin_size = 0.1
slide=0.05

list_var_decode_train_test = [('seqc_0_shape', 'shape-fixation'), ('seqc_0_shape', 'seqc_0_shape'), ('shape-fixation', 'shape-fixation')]

assert len(DFallpa["twind"].unique())==1, "not big deal. just change code below to iter over all (ev, tw)."

DFRES = decodewrapouterloop_categorical_cross_time_cross_var(DFallpa,
                                                     list_var_decode_train_test,
                                                         time_bin_size, slide, savedir=SAVEDIR, extract_params=False,
                                                            ignore_same_events=False, which_level="flex")
if len(DFRES)>0:
    decodewrapouterloop_categorical_cross_time_plot(DFRES, SAVEDIR)

M1_m (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=228
level V-2-2-0, keeping, n=186
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=233
level arcdeep-4-3-0, keeping, n=231
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=226
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 186', 'arcdeep-4-3-0 : 276', 'line-8-1-0 : 179', 'zigzagSq-1-1-1 : 228']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     186
arcdeep-4-3-0 :     276
line-8-1-0 :     179
zigzagSq-1-1-1 :     228
['var_decode', 'V-2-2-0 : 233', 'arcdeep-4-3-0 : 231', 'line-8-1-0 : 179', 'zigz

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=228
level V-2-2-0, keeping, n=186
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     12
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 186', 'line-8-1-0 : 179']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_on_grid-vartrain=s

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=228
level V-2-2-0, keeping, n=186
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 186', 'line-8-1-0 : 179']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     186
line-8-1-0 :     179
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-var

--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     12
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=sha

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=233
level arcdeep-4-3-0, keeping, n=231
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=226
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/PMv_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     12
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 233', 'line-8-1-0 : 179']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/PMv_l-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_on_grid-vart

...

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=228
level V-2-2-0, keeping, n=186
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=228
level V-2-2-0, keeping, n=186
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 186', 'arcdeep-4-3-0 : 276', 'line-8-1-0 : 179', 'zigzagSq-1-1-1 : 228']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/SMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     186
arcdeep-4-3-0 :     276
line-8-1-0 :     179
zigzagSq-1-1-1 :     228
['var_decode', 'V-2-2-0 : 186', 'arcdeep-4-3-0 : 276', 'line-8-1-0 : 179', 'zigzagSq-1-1-1 : 228']
Saved to:  /tmp/Diego-230629-decode/4

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/SMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     12
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/SMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_single-vartrain=seqc_

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=228
level V-2-2-0, keeping, n=186
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 186', 'line-8-1-0 : 179']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/preSMA_p-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     186
line-8-1-0 :     179
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/preSMA_p-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_single

preSMA_a (-0.4, 0.4) fixon_preparation fixon_preparation
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=228
level V-2-2-0, keeping, n=186
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=233
level arcdeep-4-3-0, keeping, n=231
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=226
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 186', 'arcdeep-4-3-0 : 276', 'line-8-1-0 : 179', 'zigzagSq-1-1-1 : 228']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     186
arcdeep-4-3-0 :     276
line-8-1-0 :     179
zigzagSq-1-1-1 :     228
['var_decode', 'V-2-2-0 : 233', 'arcdeep-4-3-0 : 231', 'line-8-1-0 : 179

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=228
level V-2-2-0, keeping, n=186
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     12
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 186', 'line-8-1-0 : 179']
Saved to:  /tmp/Diego-230629-decode/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation-evtest=fixon_preparation-tktrain=prims_single-tktest=prims_on_grid-va

In [8]:
# result: looks like you can decode in PMv_m, PMv_l pretty well.
# questions:
# - does effect differ for first macrosaccade vs. the rest? (e.g. no pre-fixation effect if macrosacc=1)
# - after subtracting mean (to approximately account for confound), can we still decode shapefix well?

In [9]:
# make a new pa with twice the columns: for each brain region, two "fake" events (train and test)
# each pa should have the mean subtracted.
DFallpa_mean_adj = DFallpa.copy()
DFallpa_mean_adj = pd.DataFrame(np.repeat(DFallpa_mean_adj.values, 2, axis=0))
DFallpa_mean_adj.columns = DFallpa.columns

CONJ_VARS_1 = ("shape-macrosaccade-index", "shape-fixation", "loc-fixation", "seqc_0_loc")
CONJ_VARS_2 = ("shape-macrosaccade-index", "seqc_0_shape", "seqc_0_loc", "loc-fixation")

nr = DFallpa_mean_adj.shape[0]
for i in range(0,nr):
    if i%2==0:
        event = "fixon_preparation_train"
        CONJ_VARS = CONJ_VARS_1
    else:
        event = "fixon_preparation_test"
        CONJ_VARS = CONJ_VARS_2
    pa_old = DFallpa_mean_adj.iloc[i]['pa']
    pa_new = pa_old.norm_by_label_subtract_mean("trials", CONJ_VARS)
    DFallpa_mean_adj.iloc[i]['pa'] = pa_new
    DFallpa_mean_adj.iloc[i]['event'] = event

In [10]:
DFallpa_mean_adj

,which_level,event,bregion,twind,pa,pa_x_shape
0,flex,fixon_preparation_train,M1_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(22, 896, 80)"
1,flex,fixon_preparation_test,M1_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(22, 896, 80)"
2,flex,fixon_preparation_train,M1_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(17, 896, 80)"
3,flex,fixon_preparation_test,M1_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(17, 896, 80)"
4,flex,fixon_preparation_train,PMv_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(39, 896, 80)"
5,flex,fixon_preparation_test,PMv_l,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(39, 896, 80)"
6,flex,fixon_preparation_train,PMv_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(31, 896, 80)"
7,flex,fixon_preparation_test,PMv_m,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(31, 896, 80)"
8,flex,fixon_preparation_train,PMd_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(24, 896, 80)"
9,flex,fixon_preparation_test,PMd_p,"(-0.4, 0.4)",<neuralmonkey.classes.population.PopAnal objec...,"(24, 896, 80)"


In [11]:
## Cross-temporal decoder, trained on seqc_0_shape, tested on shape-fixation

from neuralmonkey.analyses.decode_good import decodewrapouterloop_categorical_cross_time_cross_var, decodewrapouterloop_categorical_cross_time_plot

SAVEDIR_ANALYSIS = f"/tmp/{animal}-{date}-decode_mean_adj"
SAVEDIR = f"{SAVEDIR_ANALYSIS}/4_cross_temporal_diff_var_train_test"
os.makedirs(SAVEDIR, exist_ok=True)

time_bin_size = 0.1
slide=0.05

list_var_decode_train_test = [('seqc_0_shape', 'shape-fixation'), ('seqc_0_shape', 'seqc_0_shape'), ('shape-fixation', 'shape-fixation')]

DFRES = decodewrapouterloop_categorical_cross_time_cross_var(DFallpa_mean_adj,
                                                     list_var_decode_train_test,
                                                         time_bin_size, slide, savedir=SAVEDIR, extract_params=False,
                                                            ignore_same_events=False, which_level="flex")
if len(DFRES)>0:
    decodewrapouterloop_categorical_cross_time_plot(DFRES, SAVEDIR)

M1_m (-0.4, 0.4) fixon_preparation_train fixon_preparation_train
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level zigzagSq-1-1-1, keeping, n=228
level line-8-1-0, keeping, n=179
level V-2-2-0, keeping, n=186
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=233
level arcdeep-4-3-0, keeping, n=231
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=226
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 186', 'arcdeep-4-3-0 : 276', 'line-8-1-0 : 179', 'zigzagSq-1-1-1 : 228']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     186
arcdeep-4-3-0 :     276
line-8-1-0 :     179
zigzagSq-1-1-1 :     228
['var_decode', 'V-2-2-0 : 233', 'arcdeep-4-3-0 

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, REMOVING, n=4
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
(removing this many indices):  8
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level zigzagSq-1-1-1, keeping, n=228
level line-8-1-0, keeping, n=179
level V-2-2-0, keeping, n=186
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_train-tktrain=prims_single-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     12
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 186', 'line-8-1-0 : 179']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_train-tktrain=pr

--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=233
level arcdeep-4-3-0, keeping, n=231
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=226
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=233
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=226
level arcdeep-4-3-0, keeping, n=231
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 233', 'arcdeep-4-3-0 : 231', 'line-8-1-0 : 179', 'zigzagSq-1-1-1 : 226']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     233
arcdeep-4-3-0 :     231
line-8-1-0 :     179
zigzagSq-1-1-1 :     226
['var_decode', 'V-2-2-0 : 233', 'arcdeep-4-3-0 : 231', 'line-8-1-0 : 179', 'zigzagSq-1-1-1 : 226']
Saved to: 

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, REMOVING, n=4
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
(removing this many indices):  8
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, REMOVING, n=4
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     12
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_single-

--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=233
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=226
level arcdeep-4-3-0, keeping, n=231
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, REMOVING, n=4
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 233', 'line-8-1-0 : 179']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_single-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     233
line-8-1-0 :     179
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-t

...

--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=233
level arcdeep-4-3-0, keeping, n=231
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=226
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, REMOVING, n=4
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 233', 'line-8-1-0 : 179']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_single-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     233
line-8-1-0 :     179
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparatio

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=228
level V-2-2-0, keeping, n=186
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level zigzagSq-1-1-1, keeping, n=228
level line-8-1-0, keeping, n=179
level V-2-2-0, keeping, n=186
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 186', 'arcdeep-4-3-0 : 276', 'line-8-1-0 : 179', 'zigzagSq-1-1-1 : 228']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     186
arcdeep-4-3-0 :     276
line-8-1-0 :     179
zigzagSq-1-1-1 :     228
['var_decode', 'V-2-2-0 : 186', 'arcdeep-4-3-0 : 276', 'line-8-1-0 : 179', 'zigzagSq-1-1-1 : 228']
Saved to:  /tm

--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, REMOVING, n=4
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
(removing this many indices):  8
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=233
level arcdeep-4-3-0, keeping, n=231
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=226
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_single-tktest=prims_on_grid-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     12
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 233', 'line-8-1-0 : 179']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_tr

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=276
level line-8-1-0, keeping, n=179
level zigzagSq-1-1-1, keeping, n=228
level V-2-2-0, keeping, n=186
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, REMOVING, n=4
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 186', 'line-8-1-0 : 179']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     186
line-8-1-0 :     179
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test

--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, REMOVING, n=4
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
(removing this many indices):  8
--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, REMOVING, n=4
level V-2-2-0, keeping, n=12
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test-tktrain=prims_single-tktest=prims_single-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     12
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 12', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_mean_adj/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test-tktrain

In [12]:
# first macrosaccade
DFallpa_ifmt = DFallpa.copy()

nr = DFallpa_ifmt.shape[0]
for i in range(0,nr):
    pa_old = DFallpa_ifmt.iloc[i]['pa']
    pa_new = pa_old.slice_by_labels('trials', 'shape-macrosaccade-index', [1])
    DFallpa_ifmt.iloc[i]['pa'] = pa_new

SAVEDIR_ANALYSIS = f"/tmp/{animal}-{date}-decode_ifmt"
SAVEDIR = f"{SAVEDIR_ANALYSIS}/4_cross_temporal_diff_var_train_test"
os.makedirs(SAVEDIR, exist_ok=True)

DFRES = decodewrapouterloop_categorical_cross_time_cross_var(DFallpa_ifmt,
                                                     list_var_decode_train_test,
                                                         time_bin_size, slide, savedir=SAVEDIR, extract_params=False,
                                                            ignore_same_events=False, which_level="flex")
if len(DFRES)>0:
    decodewrapouterloop_categorical_cross_time_plot(DFRES, SAVEDIR)

M1_m (-0.4, 0.4) fixon_preparation_train fixon_preparation_train
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=75
level line-8-1-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=52
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 49', 'arcdeep-4-3-0 : 63', 'line-8-1-0 : 43', 'zigzagSq-1-1-1 : 57']
Saved to:  /tmp/Diego-230629-decode_ifmt/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     49
arcdeep-4-3-0 :     63
line-8-1-0 :     43
zigzagSq-1-1-1 :     57
['var_decode', 'V-2-2-0 : 75', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 49', 'arcdeep-4-3-0 : 63', 'line-8-1-0 : 43', 'zigzagSq-1-1-1 : 57']
Saved to:  /tmp/Diego-230629-decode_ifmt/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     49
arcdeep-4-3-0 :     63
line-8-1-0 :     43
zigzagSq-1-1-1 :     57
['var_decode', 'V-2-2-0 : 49', 'arcdeep-4-3-0 : 63', 'line-8-1-0 : 43', 'zigzagSq-1-1-1 : 57']
Saved to:  /tmp/Diego-230629-decode_ifmt/4

--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=75
level line-8-1-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=52
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=75
level line-8-1-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=52
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 75', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 33', 'zigzagSq-1-1-1 : 52']
Saved to:  /tmp/Diego-230629-decode_ifmt/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     75
arcdeep-4-3-0 :     52
line-8-1-0 :     33
zigzagSq-1-1-1 :     52
['var_decode', 'V-2-2-0 : 75', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 33', 'zigzagSq-1-1-1 : 52']
Saved to:  /tmp/Diego-230629-decod

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, REMOVING, n=2
(removing this many indices):  2
SKIPPING, becuase only not enough data:
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, REMOVING, n=2
(removing this many indices):  2
SKIPPING, becuase only not enough data:
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=75
level line-8-1-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=52
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixa

--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=75
level line-8-1-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=52
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 49', 'arcdeep-4-3-0 : 63', 'line-8-1-0 : 43', 'zigzagSq-1-1-1 : 57']
Saved to:  /tmp/Diego-230629-decode_ifmt/4_cross_temporal_diff_var_train_test/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     49
arcdeep-4-3-0 :     63
line-8-1-0 :     43
zigzagSq-1-1-1 :     57
['var_decode', 'V-2-2-0 : 75', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 33', 'zigzagSq-1-1-1 : 52']
Saved to:  /tmp/Diego-230629-decode_ifmt/4_cross_temporal_diff_var_train_test/M1_l-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=shape-fixation-TEST.yam

...

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 49', 'arcdeep-4-3-0 : 63', 'line-8-1-0 : 43', 'zigzagSq-1-1-1 : 57']
Saved to:  /tmp/Diego-230629-decode_ifmt/4_cross_temporal_diff_var_train_test/preSMA_p-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     49
arcdeep-4-3-0 :     63
line-8-1-0 :     43
zigzagSq-1-1-1 :     57
['var_decode', 'V-2-2-0 : 49', 'arcdeep-4-3-0 : 63', 'line-8-1-0 : 43', 'zigzagSq-1-1-1 : 57']
Saved to:  /tmp/Diego-230629-decode_if

--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=75
level line-8-1-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=52
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=75
level line-8-1-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=52
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 75', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 33', 'zigzagSq-1-1-1 : 52']
Saved to:  /tmp/Diego-230629-decode_ifmt/4_cross_temporal_diff_var_train_test/preSMA_p-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     75
arcdeep-4-3-0 :     52
line-8-1-0 :     33
zigzagSq-1-1-1 :     52
['var_decode', 'V-2-2-0 : 75', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 33', 'zigzagSq-1-1-1 : 52']
Saved to:  /tmp/Diego-230629-de

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, REMOVING, n=2
(removing this many indices):  2
SKIPPING, becuase only not enough data:
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, REMOVING, n=2
(removing this many indices):  2
SKIPPING, becuase only not enough data:
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=75
level line-8-1-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=52
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixa

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=75
level line-8-1-0, keeping, n=33
level arcdeep-4-3-0, keeping, n=52
level zigzagSq-1-1-1, keeping, n=52
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 49', 'arcdeep-4-3-0 : 63', 'line-8-1-0 : 43', 'zigzagSq-1-1-1 : 57']
Saved to:  /tmp/Diego-230629-decode_ifmt/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     49
arcdeep-4-3-0 :     63
line-8-1-0 :     43
zigzagSq-1-1-1 :     57
['var_decode', 'V-2-2-0 : 75', 'arcdeep-4-3-0 : 52', 'line-8-1-0 : 33', 'zigzagSq-1-1-1 : 52']
Saved to:  /tmp/Diego-230629-decod

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=63
level line-8-1-0, keeping, n=43
level zigzagSq-1-1-1, keeping, n=57
level V-2-2-0, keeping, n=49
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 49', 'arcdeep-4-3-0 : 63', 'line-8-1-0 : 43', 'zigzagSq-1-1-1 : 57']
Saved to:  /tmp/Diego-230629-decode_ifmt/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     49
arcdeep-4-3-0 :     63
line-8-1-0 :     43
zigzagSq-1-1-1 :     57
['var_decode', 'V-2-2-0 : 49', 'arcdeep-4-3-0 : 63', 'line-8-1-0 : 43', 'zigzagSq-1-1-1 : 57']
Saved to:  /tmp/Diego-230629-decode_ifm

In [13]:
# rest of macrosaccades
DFallpa_ifmf = DFallpa.copy()

nr = DFallpa_ifmf.shape[0]
for i in range(0,nr):
    pa_old = DFallpa_ifmf.iloc[i]['pa']
    pa_new = pa_old.slice_by_labels('trials', 'shape-macrosaccade-index', list(range(2,11)))
    DFallpa_ifmf.iloc[i]['pa'] = pa_new

SAVEDIR_ANALYSIS = f"/tmp/{animal}-{date}-decode_ifmf"
SAVEDIR = f"{SAVEDIR_ANALYSIS}/4_cross_temporal_diff_var_train_test"
os.makedirs(SAVEDIR, exist_ok=True)

DFRES = decodewrapouterloop_categorical_cross_time_cross_var(DFallpa_ifmf,
                                                     list_var_decode_train_test,
                                                         time_bin_size, slide, savedir=SAVEDIR, extract_params=False,
                                                            ignore_same_events=False, which_level="flex")
if len(DFRES)>0:
    decodewrapouterloop_categorical_cross_time_plot(DFRES, SAVEDIR)

M1_m (-0.4, 0.4) fixon_preparation_train fixon_preparation_train
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=213
level line-8-1-0, keeping, n=136
level zigzagSq-1-1-1, keeping, n=171
level V-2-2-0, keeping, n=137
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, keeping, n=179
level V-2-2-0, keeping, n=158
level line-8-1-0, keeping, n=146
level zigzagSq-1-1-1, keeping, n=174
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 137', 'arcdeep-4-3-0 : 213', 'line-8-1-0 : 136', 'zigzagSq-1-1-1 : 171']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     137
arcdeep-4-3-0 :     213
line-8-1-0 :     136
zigzagSq-1-1-1 :     171
['var_decode', 'V-2-2-0 : 158', 'arcdeep-4-3-0 : 17

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=10
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=213
level line-8-1-0, keeping, n=136
level zigzagSq-1-1-1, keeping, n=171
level V-2-2-0, keeping, n=137
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 10', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_train-tktrain=prims_single-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     10
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 137', 'line-8-1-0 : 136']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_train-tktrain=prims_sing

--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, keeping, n=179
level V-2-2-0, keeping, n=158
level line-8-1-0, keeping, n=146
level zigzagSq-1-1-1, keeping, n=174
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, keeping, n=179
level V-2-2-0, keeping, n=158
level line-8-1-0, keeping, n=146
level zigzagSq-1-1-1, keeping, n=174
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 158', 'arcdeep-4-3-0 : 179', 'line-8-1-0 : 146', 'zigzagSq-1-1-1 : 174']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     158
arcdeep-4-3-0 :     179
line-8-1-0 :     146
zigzagSq-1-1-1 :     174
['var_decode', 'V-2-2-0 : 158', 'arcdeep-4-3-0 : 179', 'line-8-1-0 : 146', 'zigzagSq-1-1-1 : 174']
Saved to:  /tm

--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=10
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
--- [filter_by_min_n], checking var= seqc_0_shape
level V-2-2-0, keeping, n=10
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 10', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_single-tktest=prims_single-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     10
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 10', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_single-tktest=p

--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, keeping, n=179
level V-2-2-0, keeping, n=158
level line-8-1-0, keeping, n=146
level zigzagSq-1-1-1, keeping, n=174
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=10
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 158', 'line-8-1-0 : 146']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_single-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     158
line-8-1-0 :     146
['var_decode', 'V-2-2-0 : 10', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/M1_m-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=p

...

--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, keeping, n=179
level V-2-2-0, keeping, n=158
level line-8-1-0, keeping, n=146
level zigzagSq-1-1-1, keeping, n=174
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=10
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 158', 'line-8-1-0 : 146']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_single-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     158
line-8-1-0 :     146
['var_decode', 'V-2-2-0 : 10', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_train-evtest=fixon_preparation_test-t

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=213
level line-8-1-0, keeping, n=136
level zigzagSq-1-1-1, keeping, n=171
level V-2-2-0, keeping, n=137
(removing this many indices):  0
--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=213
level line-8-1-0, keeping, n=136
level zigzagSq-1-1-1, keeping, n=171
level V-2-2-0, keeping, n=137
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 137', 'arcdeep-4-3-0 : 213', 'line-8-1-0 : 136', 'zigzagSq-1-1-1 : 171']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_on_grid-tktest=prims_on_grid-vartrain=seqc_0_shape-vartest=seqc_0_shape-TRAIN.yaml
V-2-2-0 :     137
arcdeep-4-3-0 :     213
line-8-1-0 :     136
zigzagSq-1-1-1 :     171
['var_decode', 'V-2-2-0 : 137', 'arcdeep-4-3-0 : 213', 'line-8-1-0 : 136', 'zigzagSq-1-1-1 : 171']
Saved to:  /tmp/Di

--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=10
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
--- [filter_by_min_n], checking var= shape-fixation
level arcdeep-4-3-0, keeping, n=179
level V-2-2-0, keeping, n=158
level line-8-1-0, keeping, n=146
level zigzagSq-1-1-1, keeping, n=174
(removing this many indices):  0
['var_decode', 'V-2-2-0 : 10', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktrain=prims_single-tktest=prims_on_grid-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     10
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 158', 'line-8-1-0 : 146']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_train-tktr

--- [filter_by_min_n], checking var= seqc_0_shape
level arcdeep-4-3-0, keeping, n=213
level line-8-1-0, keeping, n=136
level zigzagSq-1-1-1, keeping, n=171
level V-2-2-0, keeping, n=137
(removing this many indices):  0
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=10
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 137', 'line-8-1-0 : 136']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test-tktrain=prims_on_grid-tktest=prims_single-vartrain=seqc_0_shape-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     137
line-8-1-0 :     136
['var_decode', 'V-2-2-0 : 10', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test-tktrain

--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=10
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
--- [filter_by_min_n], checking var= shape-fixation
level V-2-2-0, keeping, n=10
level zigzagSq-1-1-1, REMOVING, n=4
level line-8-1-0, keeping, n=7
level arcdeep-4-3-0, REMOVING, n=4
(removing this many indices):  8
['var_decode', 'V-2-2-0 : 10', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test-tktrain=prims_single-tktest=prims_single-vartrain=shape-fixation-vartest=shape-fixation-TRAIN.yaml
V-2-2-0 :     10
line-8-1-0 :     7
['var_decode', 'V-2-2-0 : 10', 'line-8-1-0 : 7']
Saved to:  /tmp/Diego-230629-decode_ifmf/4_cross_temporal_diff_var_train_test/preSMA_a-(-0.4, 0.4)-evtrain=fixon_preparation_test-evtest=fixon_preparation_test-tktrain=prims_s